<a href="https://colab.research.google.com/github/elephanti/NLPProject2024/blob/main/Filter_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
def filter_and_analyze_csv(url, save_path='top_10_matches_df.csv', top=10):
  """
  Loads a CSV file, filters it based on matching and non-matching labels,
  and saves a CSV file with the top 10 matches per label.

  Args:
    url: The URL of the CSV file.
    save_path: The path to save the filtered CSV file.
  """

  df = pd.read_csv(url)
  # first make sure all label + text combinations are unique
  df = df.drop_duplicates(subset=['label', 'text'])

  matching_df = df[df['label'] == df['predicted_label']]
  non_matching_df = df[df['label'] != df['predicted_label']]

  # Calculate matches and non-matches per label
  matching_counts = matching_df['label'].value_counts()
  non_matching_counts = non_matching_df['label'].value_counts()

  # Create a summary DataFrame
  summary_df = pd.DataFrame({
      'Matching': matching_counts,
      'Non-Matching': non_matching_counts
  }).fillna(0).astype(int)

  # show also percentage of matching per skill
  summary_df['Matching %'] = summary_df['Matching'] / (summary_df['Matching'] + summary_df['Non-Matching']) * 100

  # sort by matching percentage
  summary_df = summary_df.sort_values('Matching %', ascending=False)
  print(summary_df)

  # print also the total filtered records
  print('Total filtered records:', len(non_matching_df))

  top_k_matches_per_label = []
  for label in summary_df.index:
    temp_df = matching_df[matching_df['label'] == label].sort_values('probability', ascending=False).head(top)
    top_k_matches_per_label.append(temp_df)

  top_k_matches_df = pd.concat(top_k_matches_per_label)
  top_k_matches_df.to_csv(save_path, index=False)
  files.download(save_path)
  return len(non_matching_df), len(matching_df)


In [ ]:
def url_builder(algo, model, dataset, classifier, subset_size):
  if model == "Llama3-Instruct":
    if algo != "Lambada+Instruct":
      return None


  if algo == "Lambada+Instruct":
    if model != "Llama3-Instruct":
      return None

  url = f"https://github.com/elephanti/NLPProject2024/raw/main/datasets_predictions/{algo}/{model}/{dataset}/{classifier}/"

  if model == "Llama3":
    url += "Llama3_8B_"

  elif model == "Llama3-Instruct":
    url += "Llama_8B_"

  elif model == "Mistral":
    url += "Mistral_7B_"

  if algo == "Lambada+":
    url += "lambada_plus_"

  elif algo == "Lambada+Instruct":
    url += "lambada_plus_instruct_"

  url += dataset
  url += f"_{subset_size}_augmented_data_"
  url += classifier
  url += ".csv"

  return url

In [ ]:
urls = []

for algo in ['Lambada+', 'Lambada+Instruct', 'Lambada']:
  for model in ['Llama3', 'Llama3-Instruct', 'Mistral']:
    for dataset in ['ATIS', 'TREC']:
      for classifier in ['bert_ktrain', 'lstm_glove', 'svm_glove', 'svm_tfidf']:
        for subset_size in [5, 10, 20, 50, 100]:
          url = url_builder(algo, model, dataset, classifier, subset_size)

          if not url:
            continue

          # try get this url
          try:
            df = pd.read_csv(url)
            urls.append(url)
          except:
            print(f"Error: {url}")


Error: https://github.com/elephanti/NLPProject2024/raw/main/datasets_predictions/Lambada+Instruct/Llama3-Instruct/TREC/bert_ktrain/Llama_8B_lambada_plus_instruct_TREC_100_augmented_data_bert_ktrain.csv
Error: https://github.com/elephanti/NLPProject2024/raw/main/datasets_predictions/Lambada+Instruct/Llama3-Instruct/TREC/lstm_glove/Llama_8B_lambada_plus_instruct_TREC_100_augmented_data_lstm_glove.csv
Error: https://github.com/elephanti/NLPProject2024/raw/main/datasets_predictions/Lambada+Instruct/Llama3-Instruct/TREC/svm_glove/Llama_8B_lambada_plus_instruct_TREC_100_augmented_data_svm_glove.csv
Error: https://github.com/elephanti/NLPProject2024/raw/main/datasets_predictions/Lambada+Instruct/Llama3-Instruct/TREC/svm_tfidf/Llama_8B_lambada_plus_instruct_TREC_100_augmented_data_svm_tfidf.csv


In [ ]:
urls

['https://github.com/elephanti/NLPProject2024/raw/main/datasets_predictions/Lambada+/Llama3/ATIS/bert_ktrain/Llama3_8B_lambada_plus_ATIS_5_augmented_data_bert_ktrain.csv',
 'https://github.com/elephanti/NLPProject2024/raw/main/datasets_predictions/Lambada+/Llama3/ATIS/bert_ktrain/Llama3_8B_lambada_plus_ATIS_10_augmented_data_bert_ktrain.csv',
 'https://github.com/elephanti/NLPProject2024/raw/main/datasets_predictions/Lambada+/Llama3/ATIS/bert_ktrain/Llama3_8B_lambada_plus_ATIS_20_augmented_data_bert_ktrain.csv',
 'https://github.com/elephanti/NLPProject2024/raw/main/datasets_predictions/Lambada+/Llama3/ATIS/bert_ktrain/Llama3_8B_lambada_plus_ATIS_50_augmented_data_bert_ktrain.csv',
 'https://github.com/elephanti/NLPProject2024/raw/main/datasets_predictions/Lambada+/Llama3/ATIS/bert_ktrain/Llama3_8B_lambada_plus_ATIS_100_augmented_data_bert_ktrain.csv',
 'https://github.com/elephanti/NLPProject2024/raw/main/datasets_predictions/Lambada+/Llama3/ATIS/lstm_glove/Llama3_8B_lambada_plus_ATIS

In [ ]:
stats = {}

for url in urls:
  print(url.split('/')[-1].split('.')[0])
  save_path = url.split('/')[-1].split('.')[0] + '_filtered.csv'
  filtered_count, matching_count = filter_and_analyze_csv(url, save_path, 10)
  stats[url.split('/')[-1].split('.')[0]] = {
      'filtered_count': filtered_count,
      'matching_count': matching_count
  }
  print("="*80)

Llama3_8B_lambada_plus_ATIS_5_augmented_data_bert_ktrain
                Matching  Non-Matching  Matching %
label                                             
airport               76            24   76.000000
capacity              60            40   60.000000
flight_time           56            44   56.000000
airfare               54            46   54.000000
quantity              46            54   46.000000
abbreviation          38            62   38.000000
ground_fare           34            66   34.000000
flight_no             29            71   29.000000
airline               28            72   28.000000
aircraft              17            83   17.000000
meal                  12            88   12.000000
restriction            9            91    9.000000
city                   5            93    5.102041
flight                 1            99    1.000000
day_name               0           100    0.000000
ground_service         0           100    0.000000
cheapest               0 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Llama3_8B_lambada_plus_ATIS_10_augmented_data_bert_ktrain
                Matching  Non-Matching  Matching %
label                                             
meal                  94             6   94.000000
distance              84            10   89.361702
capacity              88            12   88.000000
quantity              84            16   84.000000
aircraft              84            16   84.000000
abbreviation          74            26   74.000000
ground_fare           72            26   73.469388
flight_time           70            30   70.000000
airline               63            37   63.000000
airfare               61            38   61.616162
airport               57            43   57.000000
ground_service        48            52   48.000000
flight                33            67   33.000000
city                  27            73   27.000000
flight_no             20            80   20.000000
cheapest               0           100    0.000000
day_name               0

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Mistral_7B_TREC_100_augmented_data_svm_tfidf
                Matching  Non-Matching  Matching %
label                                             
ENTY:color            73             2   97.333333
NUM:count             79             3   96.341463
NUM:date              85             4   95.505618
DESC:manner           84             4   95.454545
LOC:state             85             6   93.406593
ENTY:lang             68             5   93.150685
NUM:money             73             6   92.405063
LOC:country           83             8   91.208791
NUM:period            72             8   90.000000
ENTY:instru           51             6   89.473684
ENTY:substance        78            11   87.640449
LOC:mount             77            11   87.500000
NUM:temp              61             9   87.142857
ENTY:currency         84            14   85.714286
ENTY:animal           77            13   85.555556
NUM:dist              58            10   85.294118
NUM:weight            61            1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
stats

{'Llama3_8B_lambada_plus_ATIS_5_augmented_data_bert_ktrain': {'filtered_count': 1332,
  'matching_count': 465},
 'Llama3_8B_lambada_plus_ATIS_10_augmented_data_bert_ktrain': {'filtered_count': 832,
  'matching_count': 959},
 'Llama3_8B_lambada_plus_ATIS_20_augmented_data_bert_ktrain': {'filtered_count': 731,
  'matching_count': 1034},
 'Llama3_8B_lambada_plus_ATIS_50_augmented_data_bert_ktrain': {'filtered_count': 660,
  'matching_count': 1114},
 'Llama3_8B_lambada_plus_ATIS_100_augmented_data_bert_ktrain': {'filtered_count': 660,
  'matching_count': 1119},
 'Llama3_8B_lambada_plus_ATIS_5_augmented_data_lstm_glove': {'filtered_count': 1594,
  'matching_count': 203},
 'Llama3_8B_lambada_plus_ATIS_10_augmented_data_lstm_glove': {'filtered_count': 1386,
  'matching_count': 405},
 'Llama3_8B_lambada_plus_ATIS_20_augmented_data_lstm_glove': {'filtered_count': 1460,
  'matching_count': 305},
 'Llama3_8B_lambada_plus_ATIS_50_augmented_data_lstm_glove': {'filtered_count': 1217,
  'matching_cou

In [ ]:
# download all csv files in root
!zip -r /content/filtered_csvs.zip /content/

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 23%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2024.08.05/ (stored 0%)
  adding: content/.config/logs/2024.08.05/13.23.28.842671.log (deflated 57%)
  adding: content/.config/logs/2024.08.05/13.23.06.500078.log (deflated 58%)
  adding: content/.config/logs/2024.08.05/13.22.45.846144.log (deflated 93%)
  adding: content/.config/logs/2024.08.05/13.23.17.376172.log (deflated 85%)

In [ ]:
# prompt: Arrange the filtered csvs in folders hierarchy: algorithm/ model / dataset / classifier and then zip it

import os
import shutil

# Create the directory structure
for algo in ['Lambada+', 'Lambada+Instruct', 'Lambada']:
  for model in ['Llama3', 'Llama3-Instruct', 'Mistral']:
    for dataset in ['ATIS', 'TREC']:
      for classifier in ['bert_ktrain', 'lstm_glove', 'svm_glove', 'svm_tfidf']:
        dir_path = f"/content/filtered_csvs/{algo}/{model}/{dataset}/{classifier}"
        os.makedirs(dir_path, exist_ok=True)

# Move the files
for filename in os.listdir("/content/"):
  if filename.endswith("_filtered.csv"):
    print
    if "ATIS" in filename:
      dataset = "ATIS"
    if "TREC" in filename:
      dataset = "TREC"
    if "Llama_8b" in filename:
      model = "Llama3-Instruct"
      algo = "Lambada+Instruct"
    if "Llama3_8b" in filename:
      model = "Llama3"
    if "lambada_plus" in filename:
      algo = "Lambada+"
    if "lambada" not in filename:
      algo = "Lambada"
    if "Mistral" in filename:
      model = "Mistral"
    if "bert_ktrain" in filename:
      classifier = "bert_ktrain"
    if "lstm_glove" in filename:
      classifier = "lstm_glove"
    if "svm_glove" in filename:
      classifier = "svm_glove"
    if "svm_tfidf" in filename:
      classifier = "svm_tfidf"
    if not model or not algo or not dataset or not classifier:
      raise Exception(f"Error: {filename}")

    src_path = f"/content/{filename}"
    dst_path = f"/content/filtered_csvs/{algo}/{model}/{dataset}/{classifier}/{filename}"
    shutil.move(src_path, dst_path)

# Zip the directory
!zip -r /content/filtered_csvs.zip /content/filtered_csvs


  adding: content/filtered_csvs/ (stored 0%)
  adding: content/filtered_csvs/Lambada+/ (stored 0%)
  adding: content/filtered_csvs/Lambada+/Mistral/ (stored 0%)
  adding: content/filtered_csvs/Lambada+/Mistral/TREC/ (stored 0%)
  adding: content/filtered_csvs/Lambada+/Mistral/TREC/lstm_glove/ (stored 0%)
  adding: content/filtered_csvs/Lambada+/Mistral/TREC/lstm_glove/Mistral_7B_lambada_plus_TREC_20_augmented_data_lstm_glove_filtered.csv (deflated 69%)
  adding: content/filtered_csvs/Lambada+/Mistral/TREC/lstm_glove/Llama_8B_lambada_plus_instruct_TREC_10_augmented_data_lstm_glove_filtered.csv (deflated 61%)
  adding: content/filtered_csvs/Lambada+/Mistral/TREC/lstm_glove/Mistral_7B_lambada_plus_TREC_5_augmented_data_lstm_glove_filtered.csv (deflated 70%)
  adding: content/filtered_csvs/Lambada+/Mistral/TREC/lstm_glove/Llama_8B_lambada_plus_instruct_TREC_20_augmented_data_lstm_glove_filtered.csv (deflated 62%)
  adding: content/filtered_csvs/Lambada+/Mistral/TREC/lstm_glove/Llama3_8B_la